# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\tang\AppData\Local\Temp\graphlab_server_1496664318.log.0


This non-commercial license of GraphLab Create for academic use is assigned to 1962788480@qq.com and will expire on April 16, 2018.


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe=data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray=data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions=np.dot(feature_matrix,weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [9]:
print 1

1


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [10]:
def normalize_features(feature_matrix):
    norms=np.linalg.norm(feature_matrix,axis=0)
    normalized_features=feature_matrix/norms
    return (normalized_features, norms)

To test the function, run the following:

In [11]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [13]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [14]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [15]:
prediction = predict_output(simple_feature_matrix,weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [16]:
r=[]
r.append((simple_feature_matrix[:,0]*(output-prediction+weights[0]*simple_feature_matrix[:,0])).sum())
r.append((simple_feature_matrix[:,1]*(output-prediction+weights[1]*simple_feature_matrix[:,1])).sum())
r.append((simple_feature_matrix[:,2]*(output-prediction+weights[2]*simple_feature_matrix[:,2])).sum())

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [17]:
print r

[79400300.034929156, 87939470.772991076, 80966698.675965652]


In [18]:
print 2*r[2],2*r[1]

161933397.352 175878941.546


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [19]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i =(feature_matrix[:,i]*(output-prediction+weights[i]*feature_matrix[:,i])).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i +l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i -l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [20]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [21]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = np.copy(initial_weights)
    while True:
        thres=0
        for i in range(len(weights)):
            old_weights_i = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)

            change=abs(old_weights_i-weights[i])
            #print change
            if thres<change:
                thres=change
                #print 'thres is '+str(thres)
        print 'maxthres is '+str(thres)
        if thres<tolerance:
            break
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [22]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [23]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [24]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

maxthres is 79400304.6581
maxthres is 9138168.3665
maxthres is 8194809.40937
maxthres is 6598905.03827
maxthres is 5522173.1943
maxthres is 4621129.81031
maxthres is 3867108.10625
maxthres is 3236118.81061
maxthres is 2708086.93955
maxthres is 2266213.1094
maxthres is 1896439.06265
maxthres is 1587000.40319
maxthres is 1328052.31095
maxthres is 1111356.328
maxthres is 930018.251241
maxthres is 778268.79269
maxthres is 651280.029038
maxthres is 545011.800818
maxthres is 456083.174344
maxthres is 381664.876995
maxthres is 319389.283634
maxthres is 267275.090397
maxthres is 223664.279321
maxthres is 187169.368346
maxthres is 156629.268444
maxthres is 131072.343461
maxthres is 109685.497423
maxthres is 91788.3058116
maxthres is 76811.3677894
maxthres is 64278.190664
maxthres is 53790.0302253
maxthres is 45013.2046616
maxthres is 37668.4784415
maxthres is 31522.1784089
maxthres is 26378.7594496
maxthres is 22074.5832053
maxthres is 18472.7119037
maxthres is 15458.5516701
maxthres is 12936.2

In [25]:
prediction=predict_output(normalized_simple_feature_matrix,weights)
error=prediction-output
rss=(error*error).sum()
print rss

1.63049248148e+15


In [26]:
print weights

[ 21624998.36636292  63157246.78545421         0.        ]


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [27]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [28]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [29]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
(normalized_feature_matrix, norms) = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [30]:
l1_penalty=1e7
tolerance=1
initial_weights=np.zeros(len(all_features)+1)
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

maxthres is 71114625.7528
maxthres is 5024356.36869
maxthres is 5012324.91395
maxthres is 5000676.42086
maxthres is 4991243.17314
maxthres is 4984361.34335
maxthres is 4979504.92921
maxthres is 4973806.56525
maxthres is 2879222.99938
maxthres is 2706150.76622
maxthres is 2532295.64864
maxthres is 2368316.57782
maxthres is 2213124.35067
maxthres is 1552443.60013
maxthres is 507838.033159
maxthres is 438814.370591
maxthres is 371479.622608
maxthres is 313415.105581
maxthres is 264244.832139
maxthres is 222753.8608
maxthres is 187770.667306
maxthres is 158280.093357
maxthres is 133420.901606
maxthres is 112465.988296
maxthres is 94802.2134519
maxthres is 79912.6874505
maxthres is 67361.6926196
maxthres is 56781.942492
maxthres is 47863.835752
maxthres is 40346.3966195
maxthres is 34009.6378518
maxthres is 28668.1231465
maxthres is 24165.5406129
maxthres is 20370.1285268
maxthres is 17170.8195088
maxthres is 14473.990295
maxthres is 12200.7219837
maxthres is 10284.4905855
maxthres is 8669.

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [31]:
print weights1e7

[ 24429600.60933314         0.                 0.          48389174.35227978
         0.                 0.           3317511.16271982   7329961.9848964
         0.                 0.                 0.                 0.
         0.                 0.        ]


Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [32]:
l1_penalty=1e8
tolerance=1
initial_weights2=np.zeros(len(all_features)+1)
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights2, l1_penalty, tolerance)

maxthres is 71114625.7528
maxthres is 0


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [33]:
print weights1e8

[ 71114625.75280938         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]


Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [39]:
l1_penalty=1e4
tolerance=5e5
initial_weights3=np.zeros(len(all_features)+1)
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,initial_weights3, l1_penalty, tolerance)

maxthres is 71114625.7528
maxthres is 8464015.94738
maxthres is 6818451.35869
maxthres is 5570165.21901
maxthres is 4559838.46759
maxthres is 3732056.69715
maxthres is 3053211.61353
maxthres is 2497517.43558
maxthres is 2043868.97325
maxthres is 1674717.70906
maxthres is 1375420.75692
maxthres is 1296621.8143
maxthres is 1257340.9361
maxthres is 1228271.24819
maxthres is 1206854.36471
maxthres is 1191059.40598
maxthres is 1179030.77017
maxthres is 1169777.43345
maxthres is 1162340.67589
maxthres is 1155980.52901
maxthres is 1150139.67966
maxthres is 1144406.32752
maxthres is 1138483.91805
maxthres is 1132166.57169
maxthres is 1125319.16778
maxthres is 1117861.24888
maxthres is 1109754.06282
maxthres is 1100990.17907
maxthres is 1091585.21244
maxthres is 1081571.26761
maxthres is 1070991.78405
maxthres is 1059897.51701
maxthres is 1048343.43631
maxthres is 1036386.3636
maxthres is 1024083.20068
maxthres is 1011489.62847
maxthres is 998659.178392
maxthres is 985642.596017
maxthres is 972

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [40]:
print weights1e4

[ 77779073.91265225 -22884012.25023361  15348487.08089996
  92166869.69883074  -2139328.0824278   -8818455.54409492
   6494209.73310655   7065162.05053198   4119079.21006765
  18436483.52618776 -14566678.54514342  -5528348.75179426
 -83591746.20730537   2784276.46012858]


## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [41]:
normalized_weights1e4=weights1e4/norms
normalized_weights1e7=weights1e7/norms
normalized_weights1e8=weights1e8/norms
print normalized_weights1e7[3]

161.317456248


To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [36]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [42]:
prediction=predict_output(test_feature_matrix,normalized_weights1e4)
error=prediction-test_output
rss=(error*error).sum()
print rss

2.2778100476e+14


In [37]:
prediction=predict_output(test_feature_matrix,normalized_weights1e7)
error=prediction-test_output
rss=(error*error).sum()
print rss

2.75962079909e+14


In [38]:
prediction=predict_output(test_feature_matrix,normalized_weights1e8)
error=prediction-test_output
rss=(error*error).sum()
print rss

5.37166150034e+14


***QUIZ QUESTION***

Which model performed best on the test data?